# Meter Simulator
This notebook keeps generating values for a power meter, and sends them through a RabbitMQ channel with a frequency.

In [ ]:
%reload_ext autoreload
%autoreload 2
from meter import Meter ## Meter class
import pika 
from datetime import datetime as dt
import shared_params as params

## Setup Parameters
Set the meter parameters

In [ ]:
random_seed = None        # random seed for reproducible results
meter_range = (0,9000)    # the range for the power meter.

## Initialize Meter

In [ ]:
meter = Meter( 
    random_seed=random_seed,
    meter_range=meter_range
)

In [ ]:
meter.process_meter(dt.now().strftime(params.datetime_format))